In [1]:
from tdc.single_pred import Tox
import deepchem
import xgboost
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.EState import EState

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [17]:
benchmark = group.get('LD50_Zhu') 
name = benchmark['name']
train_val, test = benchmark['train_val'], benchmark['test']

firstMol = Chem.MolFromSmiles(train_val.iloc[:,1].to_list()[0])
secondMol = Chem.MolFromSmiles(train_val.iloc[:,1].to_list()[1])

#np.stack(np.array(EState.EStateIndices(firstMol)))

print(Fingerprinter.FingerprintMol(firstMol)[1])
print(Fingerprinter.FingerprintMol(secondMol)[1].__len__())

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         7.54541667
 0.         2.29647203 0.         0.         1.86946145 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         3.79175926 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 4.43847258 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         3.30841801 0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
79


In [23]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
xgb_reg = xgboost.XGBRegressor()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    trainMol = train.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    validMol = valid.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    train_valMol = train_val.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost
    trainFeat = np.stack(np.array(trainMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    validFeat = np.stack(np.array(validMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    testFeat = np.stack(np.array(testMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    train_valFeat = np.stack(np.array(train_valMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))

    #valid_data = list(zip(ecfp_f_valid, valid.iloc[:,2].to_list()))
    #valid_data = [(ecfp_f_train, train.iloc[:,2].to_list()), (ecfp_f_valid, valid.iloc[:,2].to_list())]

    #fit xgboost model and store np ndarray in xgb_pred
    xgb_reg.fit(X=train_valFeat, y=train_val.iloc[:,2], eval_metric="mae", verbose=True)
    xgb_pred = xgb_reg.predict(X=testFeat)

    # store test predictions in y_pred_test
    y_pred_test = xgb_pred

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:02<00:00, 2903.83it/s]
generating training, validation splits...


predictions for run #
1
[1.7835815 2.336146  3.3699815 3.620692  4.0475807]


100%|██████████| 5907/5907 [00:01<00:00, 3309.84it/s]
generating training, validation splits...


predictions for run #
2
[1.7835815 2.336146  3.3699815 3.620692  4.0475807]


100%|██████████| 5907/5907 [00:01<00:00, 3073.95it/s]
generating training, validation splits...


predictions for run #
3
[1.7835815 2.336146  3.3699815 3.620692  4.0475807]


100%|██████████| 5907/5907 [00:01<00:00, 3242.88it/s]
generating training, validation splits...


predictions for run #
4
[1.7835815 2.336146  3.3699815 3.620692  4.0475807]


100%|██████████| 5907/5907 [00:01<00:00, 3575.54it/s]


predictions for run #
5
[1.7835815 2.336146  3.3699815 3.620692  4.0475807]
Prediction List:
[{'ld50_zhu': array([1.7835815, 2.336146 , 3.3699815, ..., 2.006633 , 2.3742006,
       2.468544 ], dtype=float32)}, {'ld50_zhu': array([1.7835815, 2.336146 , 3.3699815, ..., 2.006633 , 2.3742006,
       2.468544 ], dtype=float32)}, {'ld50_zhu': array([1.7835815, 2.336146 , 3.3699815, ..., 2.006633 , 2.3742006,
       2.468544 ], dtype=float32)}, {'ld50_zhu': array([1.7835815, 2.336146 , 3.3699815, ..., 2.006633 , 2.3742006,
       2.468544 ], dtype=float32)}, {'ld50_zhu': array([1.7835815, 2.336146 , 3.3699815, ..., 2.006633 , 2.3742006,
       2.468544 ], dtype=float32)}]


In [24]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'ld50_zhu': {'mae': 0.673}}
{'ld50_zhu': {'mae': 0.673}}
{'ld50_zhu': {'mae': 0.673}}
{'ld50_zhu': {'mae': 0.673}}
{'ld50_zhu': {'mae': 0.673}}


{'ld50_zhu': [0.673, 0.0]}